In [1]:
# Pytorch has a capability of automatic  gradient calculation !
# In this Notebook We will learn each and every thing about autograd !

#################### Auto Grad ######################################
# Why we require auto grad !
""" 
When we do backpropragation we need to calculate gradient of loss function w.r.t weigth 
If we do gradient calculation with hands it will take time and it wont be dynamic as then we would have to write  
each derivative manually. To resolve this issue pytorch has a capability to calculate derivative of function automatically
which is also known as Auto Grad.  

"""
import torch 
from torch.autograd import grad 
import torch.nn as nn 

#### A simplified model of a PyTorch tensor is as an object containing the following properties:
1. data — a self-reference (per the above).
2. required_grad — whether or not this tensor is/should be connected to the computational graph.
3. grad — if required_grad is true, this prop will be a sub-tensor that collects the gradients against this tensor accumulated during backwards().
4. grad_fn — This is a reference to the most recent operation which generated this tensor. PyTorch performs automatic differentiation by looking through the grad_fn list.
5. is_leaf — Whether or not this is a leaf node.

# Simple Derivative 

In [2]:
# Lets Take an simple Example 
x = torch.tensor(5.0,requires_grad=True)
y = x**2
y

tensor(25., grad_fn=<PowBackward0>)

In [3]:
# Lets calculate Gradient by hand 
#  dy/dx = 2*x --- > 2x5 = 10
y.backward()
x.grad

tensor(10.)

# Partial Derivative

In [4]:
# Now lets apply Partial derivative 
x = torch.tensor(5.0,requires_grad=True)
y = torch.tensor(5.0,requires_grad=True)

f = x**2 + y**2

f.backward()
# df/dx = 2*x --- > 2*5 =10
# df/dy = 2*y --- > 2*5 =10

print(f.grad_fn)
print(x.grad)
print(y.grad)

# Here x , y Does not depend on each other they are in addition 
# So x has it's value independent of y and vice versa but 
# What if they are in multiply ?

tensor(10.)
tensor(10.)


In [5]:
x = torch.tensor(5.0,requires_grad=True)
y = torch.tensor(5.0,requires_grad=True)

f = x**2 * y**2

f.backward()
# df/dx = 2*x*y^2 --- > 2*5*25 = 250
# df/dy = 2*y*x^2 --- > 2*5*25 = 250

print(f.grad_fn)
print(x.grad)
print(y.grad)

tensor(250.)
tensor(250.)


# Nth derivative 

In [6]:
# Lets Do the double derivative 

In [7]:
def nth_derivative(f, wrt, n):

    for i in range(n):
        grads = grad(f, wrt, create_graph=True)[0]
        f = grads.sum()
        
    return grads

x = torch.tensor(5.0,requires_grad=True)

f = x**2 + x**3
print(nth_derivative(f=f, wrt=x, n=2))

tensor(32., grad_fn=<AddBackward0>)


# Derivative of a tensor 

In [8]:
# Derivative On A Tensor
# This will give you error as gradient is only constructed for scaler values 
x = torch.tensor([5.0,4.0,3.0],requires_grad=True)

f = x**2

f.backward()

x.grad

RuntimeError: ignored

In [9]:
# In order to calculate gradient of tensor we need to convert them in scaler 
x = torch.tensor([5.0,4.0,3.0],requires_grad=True)

f = x**2

# you need to add sum() to aggrigate all values  
f.sum().backward() 

x.grad

tensor([10.,  8.,  6.])

In [10]:
f.sum()

tensor(50., grad_fn=<SumBackward0>)

In [11]:
x = torch.tensor([5.0,4.0,3.0],requires_grad=True)
y = torch.tensor([5.0,4.0,3.0],requires_grad=True)

f = (x**2)*(y)

# you need to add sum() to aggrigate all values  
f.sum().backward() 

y.grad

tensor([25., 16.,  9.])

# Actual Implementation Of Back propagation 

In [16]:
# Acutal BackPropagation Mimic 
## Weights and bias of a network 
w1 = torch.tensor(3.0,requires_grad=True)
w2 = torch.tensor(5.0,requires_grad=True)
bias = torch.tensor(1.0,requires_grad=False)

## Input 
x1 = torch.tensor(3.0,requires_grad=False)
x2 = torch.tensor(5.0,requires_grad=False)

## equation 
y = w1*x1 + w2*x2 + bias

## Activation function 
### try changing it with Sigmoid 
z = nn.ReLU()(y)

## back propagation 

z.backward()

##
print(w1.grad)
print(w2.grad)



tensor(3.)
tensor(5.)


# Complex Implementation Of Back propagation 

In [21]:
# Weights 
weights = torch.tensor([[1.0,2.0,3.0],[2.0,3.0,4.0],[5.0,2.0,1.0]],requires_grad=True)
weights

tensor([[1., 2., 3.],
        [2., 3., 4.],
        [5., 2., 1.]], requires_grad=True)

In [24]:
bias = torch.ones((3,3),requires_grad=False)
bias

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [29]:
# inputs 
inputs = torch.randn((3,3),requires_grad=False)
inputs

tensor([[ 0.2509, -1.3723,  0.8958],
        [ 0.2216,  0.2017,  2.0475],
        [ 1.5091,  1.0168, -1.1928]])

In [32]:
# Here we need to do dot product of 2D tensors 

y = weights*inputs + bias
y

tensor([[ 1.2509, -1.7446,  3.6875],
        [ 1.4432,  1.6051,  9.1901],
        [ 8.5455,  3.0337, -0.1928]], grad_fn=<AddBackward0>)

In [33]:
# Activation Function 
z = nn.ReLU()(y)
z

tensor([[1.2509, 0.0000, 3.6875],
        [1.4432, 1.6051, 9.1901],
        [8.5455, 3.0337, 0.0000]], grad_fn=<ReluBackward0>)

In [34]:
## back propagation 
### Refer Tensor derivative section 
z.backward()

RuntimeError: ignored

In [35]:
# Derivative calculation 
z.sum().backward()

In [36]:
## Gradients 
print(weights.grad)


tensor([[0.2509, -0.0000, 0.8958],
        [0.2216, 0.2017, 2.0475],
        [1.5091, 1.0168, -0.0000]])
